# Inner / Outer prepare

<!-- We can write the second quantized electronic structure Hamiltonian as

$$
\begin{aligned}
V &= \frac{1}{2}\sum_l^L\left(\sum_\sigma \sum_{pq} W_{pq}^{(l)} a_{p\sigma}^\dagger a_{q\sigma}\right)^2 \\
  &= \frac{1}{2}\sum_l^L U_l \left(\sum_{\sigma}\sum_{p}^{N/2} f_p^{(l)}n_{p\sigma}\right)^2 U_l^{\dagger},
\end{aligned}
$$
where the $U_l$ are the unitary matrices that diagonalize $W^{l}$.

After some algebra, we can write $V$ in the Jordan Wigner representation as

$$
V = \frac{1}{8}\sum_l U_l \left(\sum_\sigma \sum_p^{N/2} f_p Z_{p\sigma}\right)^2 U_l^\dagger. 
$$

As before we need to define SELECT and PREPARE circuits for this particular factorization. -->

When performing state preparation on a factorized Hamiltonian that involves a sum over an auxiliary index $l$ of size $L$ followed by summing over a sum of squared terms it is common to split these into an outer and an inner prepare, similar to an outer and inner `for' loop in regular programming.

For example, for the single-factorized Hamiltonian our outer prepare prepares the state

$$
\mathrm{OuterPrepare}|0\rangle^{\otimes \lceil\log (L+1)\rceil} = \frac{1}{\sqrt{\lambda}}\left(|0\rangle \sqrt{2\sum_{pq}T_{pq}'} +  \sum_{l=1}^L |l\rangle \sum_{p\le q}|W_{pq}^{(l)}|\right) 
$$
where
$$
\lambda = \sum_{pq}^{N/2} T'_{pq} + \frac{1}{4}\sum_{l=1}^{L}\left(\sum_{pq}^{N/2} |W_{pq}^{(l)}|\right)^2 
$$

Recall that cirq-ft provides a generic state preparation routine that uses alias sampling to prepare 

$$
\sum_{\ell=0}^{L-1} \sqrt{p_\ell} |\ell\rangle |\mathrm{temp}_\ell\rangle
$$
where $p_l$ are probabilities, i.e. $\sum_l p_l = 1$. `StatePreparationAliasSampling` provides a convenient factory method `from_lcu_probs` which will take our (unnormalized) coefficients as input as well as a desired precision with which to represent these amplitudes. Thus, it remains to simply build a flattened array containing the appropriate (positive) coefficients. 

In [75]:
import numpy as np
import cirq
import cirq_ft.infra.testing as cq_testing
from cirq_ft.algos.state_preparation import StatePreparationAliasSampling
from cirq_ft.algos.state_preparation_test import construct_gate_helper_and_qubit_order 
from cirq_ft.infra.jupyter_tools import display_gate_and_compilation

np.random.seed(7)
nmo = 2
naux = 2 * nmo
Tpq = np.random.random((nmo, nmo))
Wlpq = np.random.random((naux, nmo, nmo))
c_0 = [np.sum(np.abs(Tpq))]
# ignore p <= q condition for the moment for simplicity
c_l = np.einsum("lpq->l", np.abs(Wlpq))
mu = 3
coeffs = np.concatenate((c_0, c_l))
coeffs /= sum(coeffs)
assert len(coeffs) == 1 + naux

outer_prepare = StatePreparationAliasSampling.from_lcu_probs(coeffs/np.sum(coeffs), probability_epsilon=2**(-mu) / len(coeffs))

Now test that it is correct

In [76]:
g, qubit_order, decomposed_circuit = construct_gate_helper_and_qubit_order(outer_prepare)
print(f"num qubits = {len(g.all_qubits)}")

num qubits = 26


In [77]:
result = cirq.Simulator(dtype=np.complex128).simulate(
    decomposed_circuit, qubit_order=qubit_order
)
sv = result.final_state_vector

In [78]:
# little utility function ripped from alias sampling test to get non-garbage register amplitudes
def get_prepared_state(gate: "GateHelper", state_vector: np.ndarray, num_terms: int):
    logL = len(g.quregs['selection'])
    L = num_terms
    state_vector = state_vector.reshape(2**logL, len(state_vector) // 2**logL)
    num_non_zero = (abs(state_vector) > 1e-6).sum(axis=1)
    prepared_state = state_vector.sum(axis=1)
    assert all(num_non_zero[:L] > 0) and all(num_non_zero[L:] == 0)
    assert all(np.abs(prepared_state[:L]) > 1e-6) and all(np.abs(prepared_state[L:]) <= 1e-6)
    prepared_state = prepared_state[:L] / np.sqrt(num_non_zero[:L])
    return prepared_state

In [79]:
prepared_state = get_prepared_state(g, sv, naux+1)
print(f"<psi|psi> = {sum(abs(prepared_state)**2.0)}")
print(abs(prepared_state)**2)
print(coeffs)

<psi|psi> = 0.9999999999999994
[0.2   0.225 0.225 0.175 0.175]
[0.20966678 0.2171009  0.23389351 0.17086428 0.16847453]


Now for the inner prepare we have 

$$
(\mathrm{InnerPrepare})(\mathrm{OuterPrepare})|0\rangle|\mathrm{temp}\rangle = \frac{1}{\sqrt{\lambda}}\left(
    |0\rangle \sum_{p \le q} \sqrt{2 |T'_{pq}|}|p,q\rangle + \sum_l |l\rangle \sqrt{\sum_{rs} |W_{rs}^{(l)}|}\sum_{p\le q} \sqrt{|W_{pq}^{(l)}}|p,q\rangle
\right)
$$
where we have ignored the sign-flagging-bits $|\theta_{pq}^{(l)}\rangle$.

It may be easier to first consider preparing the state

$$
IO|0\rangle = \sum_l |l\rangle\sqrt{\sum_q|W_q^{(l)}|}\sum_{q}\sqrt{W_p^{(l)}}|p\rangle
$$